In [ ]:
#import libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, Comment
import time
pd.options.display.max_rows = 10000
pd.options.display.max_seq_items = 10000
pd.set_option('display.max_colwidth', None)
from selenium import webdriver
import html5lib
import lxml
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time


### Campany Scrape

Company Scrape The first step is to pull companies that have a top work-life balance for a role you are interested in. For example, pull companies with a work-life balance >= 3.5 stars for Analyst roles. This scrape will pull a max of ~9,900 companies as Glassdoor has a bug that doesn't show more companies after page 1,000 on the company explore section of the site.

In [ ]:
#create temp df
df = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
length = 1
i = 1
#while statement will go to 1000 as Glassdoor doesn't share companies
#after page 1000
while (i <= 1000):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    
    #scrape companies with DS jobs with WLB > 4.5 stars
    #url = "https://www.glassdoor.com/Explore/browse-companies.htm?overall_rating_low=4.5&page=" + str(i) + "&occ=Data%20Scientist&filterType=RATING_WORK_LIFE_BALANCE"
    
    #scrape companies with Analyst jobs with WLB > 3.5 stars
    url = "https://www.glassdoor.com/Explore/browse-companies.htm?overall_rating_low=3.5&page=" + str(i) + "&occ=Analyst&filterType=RATING_WORK_LIFE_BALANCE"
    
    #scrape companies with overall WLB > 4.5 stars
    #url = "https://www.glassdoor.com/Explore/browse-companies.htm?overall_rating_low=4.5&page=" + str(i) + "&filterType=RATING_WORK_LIFE_BALANCE"
    
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    #table = soup.find_all('div', {'class':'col-md-8'})
    #table = soup.find_all('div', {'class':'mt-0 mb-std p-std css-1ax1pfu css-errlgf'})
    #table = soup.find_all('div', {'class':'row d-flex flex-wrap'})


    
    table = soup.find_all('h2', {'data-test':'employer-short-name'})
    #job = soup.find_all('a',{'data-test':'cell-Jobs-url'})
    job = soup.find_all('a',{'data-test':'cell-Jobs-url'})
    l=[]
    for link in soup.find_all('a',{'data-test':'cell-Jobs-url'}):
     ##   print(link.get('href'))
        l.append(link.get('href')) 
    employees = soup.find_all('span',{'data-test':'employer-size'})
    #print(employees)
    #print(job)
    length=len(table)
    #if (len(table) == 0):
    #    break;

    #print(table)
    y = pd.DataFrame(l)
    #print(y)
    x = pd.DataFrame(table)
    z = pd.DataFrame(employees)
    df = df.append(x)
    df2 = df2.append(y)
    df3 = df3.append(z)
    #print(len(df3))
    #print(i)
    i = i + 1


In [ ]:
#clean up pulled data
df.columns=['company']
df2.columns=['job_url']
df3.columns=['employees']
df4 = pd.concat([df,df2,df3],axis=1, ignore_index=True)
#df3 = pd.DataFrame(df,df2)
df4.columns=['company','job_url','employees']
df4=df4.reset_index()

In [ ]:
#saving checkpoint - save list of companies pulled
df4.to_csv('top_worklife_companies_analyst_09162023.csv')

In [ ]:
#load in saved file
df4 = pd.read_csv('top_worklife_companies_analyst_09162023.csv')

In [ ]:
df4.shape

### Job Scrape

Job Scrape from pulled companies From each of the companies scraped, the scraper will go to their company site and pull their current job openings. A call out here is that if the company has more than 40 job openings, only the first 40 jobs will be pulled as glassdoor limits the amount of jobs shown on the company profile page.

In [ ]:
df5 = pd.DataFrame()
df6 = pd.DataFrame()
#df7 = pd.DataFrame()
company_list = []
j= 1
length = 1
for i in range(0,len(df4)):

    x = df4['job_url'][i]
    x = x[:-4]
    x = x.split("-Jobs-")

    job_url='https://www.glassdoor.com' + x[0] + "-Remote-Jobs-" + x[1] + "_P" + str(j) + ".htm"
   # print(job_url)
    company = df4['company'][i]
  #  print(company)

    driver = webdriver.Chrome(ChromeDriverManager().install())#, chrome_options=options)
    driver.get(job_url)

    html = driver.page_source
    #print(html)
    soup = BeautifulSoup(html, "lxml")
   # print(soup)
    job_title = soup.find_all('div', {'class':'job-title'})
    job_location = soup.find_all('div',{'class':'css-k7ss21'})
    #job = soup.find_all('a',{'data-test':'cell-Jobs-url'})
    #job = soup.find_all('a',{'data-test':'cell-Jobs-url'})
    l=[]
    for link in soup.find_all('a',{'data-test':'job-link'}):
    #    print(link.get('href'))
        l.append(link.get('href')) 
        company_list.append(company)

    #print(job)
    #print(job_title)
    length=len(job_title)
   # print(length)
    if (len(job_title) == 0):
        j = 1
    else:
        j = j + 1

    #print(table)
    y = pd.DataFrame(l)
    #print(y)
    x = pd.DataFrame(job_title)
    z = pd.DataFrame(job_location)
    #employee_cnt = pd.DataFrame()
    df5 = df5.append(x)
    df6 = df6.append(y)


In [ ]:
#company_list
df6.shape

In [ ]:
#clean up job dataframe pulled
df5.columns=['job name']
df6.columns=['job link']
df7 = pd.concat([df5,df6],axis=1, ignore_index=True)
#df3 = pd.DataFrame(df,df2)
df7.columns=['job_name','job_link']
df7=df7.reset_index()
df7['company'] = company_list

In [ ]:
df8 = pd.merge(df7, df4, how = 'inner', on = 'company')
df8 = df8.drop(columns =['index_y','index_x','job_url'])
#df8

In [ ]:
#filter out jobs to only include certain keywords in job title
df9 = df8[(df8['job_name'].str.contains('Analyst'))|
         (df8['job_name'].str.contains('Data')) |
         (df8['job_name'].str.contains('Scientist'))]

df9.shape

In [ ]:
df9= df9.reset_index()

In [ ]:
#saving checkpoint - save jobs df
df9.to_csv('job_scrape_job_links_analyst_09162023_p1.csv')

In [ ]:
df9 = pd.read_csv('job_scrape_job_links_analyst_09162023_p1.csv')

In [ ]:
df9.head()

### Examine Job Description

Examine jobs scraped and filter out those that don't fit your search criteria and see which of those are open to WFH policy.

In [ ]:
#df9 = pd.DataFrame()
temp_l = []
#might need to run this iteratively depending on the amount of jobs 
#you pulled and the memory on your pc. 
#Keeping it iteration to around 1000 jobs is recommended
df10 = df9.head(1102)
for i in range(0,len(df10)):
    
    print(i)
    job_url2 = 'http://glassdoor.com' + df10['job_link'][i]
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(job_url2)

    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    #job_desc = soup.find_all('div', {'class':'desc css-58vpdc ecgq1xb5'})
    job_desc = soup.find_all('div', {'class':'desc css-c43yjn ecgq1xb5'})

    #print(job_desc)
    #if (job_desc.isempty()):
    job_desc2 = soup.find_all('div', {'class':'css-15k9b95 elvm8840'})

    #job_desc = soup.find_all('div', {'class':'css-15k9b95 elvm8840'})
    #print(job_desc)
    remote_check = str(job_desc).find('remote')
    remote_check2 = str(job_desc).find('WFH')
    remote_check3 = str(job_desc).find('home')
    remote_check4 = str(job_desc2).find('remote')
    remote_check5 = str(job_desc2).find('WFH')
    remote_check6 = str(job_desc2).find('home')
        
    if ((remote_check > 0) | 
        (remote_check2 > 0 ) |
        (remote_check3 > 0) |
        (remote_check4 > 0 ) |
        (remote_check5 > 0) |
        (remote_check6 > 0) 
       ):
        remote_final = 1
    else:
        remote_final = -1
    #temp_df = pd.DataFrame(remote_final)

    temp_l.append(remote_final)
    #df9 = df9.append(temp_df)
    
    
    

#df9.columns = ['remote_check']
df10['remote_check'] = temp_l#df9['remote_check']
df10.to_csv('job_scrape_final_09122023_p6.csv')


In [ ]:
#END